In [ ]:
!kill -9 $(lsof -t -i:8000)

In [ ]:
# Install dependencies
!pip install fastapi uvicorn nest_asyncio pyngrok joblib

# Imports
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import joblib
from pyngrok import ngrok
import threading
from typing import Optional

# Apply nest_asyncio for Colab
nest_asyncio.apply()

# Set your ngrok authtoken directly
ngrok.set_auth_token("32SYhsp4jWb7OVnPhhQGOxeCgbx_2AC8fxoLxV4k4QiNz9kw2")

# Load your saved pipeline (must be fitted pipeline)
pipeline = joblib.load("/content/drive/MyDrive/sales_model_pipeline2.pkl")

# Initialize FastAPI
app = FastAPI(title="Sales Prediction API")

# Input schema
class StoreWeekInput(BaseModel):
    Sales_Lag1: float
    Sales_Lag2: float
    Sales_MA3: float
    Sales_MA7: float
    Fuel_Price: float
    Temperature: float
    CPI: float
    Week: int
    Month: int
    Day: int
    IsHoliday_x: int

# --------------------------
# Prediction endpoint
# --------------------------
@app.post("/predict")
def predict_sales(data: StoreWeekInput):
    try:
        input_df = pd.DataFrame([data.dict()])
        print("DEBUG: Input DF columns:", input_df.columns.tolist())
        print("DEBUG: Input DF values:", input_df.values.tolist())

        pred = pipeline.predict(input_df)
        return {"Predicted_Sales": float(pred[0])}
    except Exception as e:
        import traceback
        return {"error": str(e), "trace": traceback.format_exc()}

from typing import Optional

# Schema for what-if request
class WhatIfInput(StoreWeekInput):
    fuel_increase_pct: float = 0.0        # % change for fuel
    markdown_increase_pct: float = 0.0    # % change for markdown (applied to Sales_MA3)
    toggle_holiday: Optional[int] = None  # force holiday ON/OFF

# What-if endpoint
@app.post("/whatif")
def whatif_sales(data: WhatIfInput):
    try:
        input_df = pd.DataFrame([data.dict()])

        # Base prediction
        base_pred = float(pipeline.predict(input_df)[0])

        # Copy for scenario
        scenario_df = input_df.copy()

        # Apply % adjustments
        if data.fuel_increase_pct != 0:
            scenario_df["Fuel_Price"] *= (1 + data.fuel_increase_pct / 100.0)

        if data.markdown_increase_pct != 0:
            scenario_df["Sales_MA3"] *= (1 + data.markdown_increase_pct / 100.0)

        # Optional: also scale Sales_MA7 a bit if markdown applied
        if data.markdown_increase_pct != 0:
            scenario_df["Sales_MA7"] *= (1 + (data.markdown_increase_pct * 0.8) / 100.0)

        # Toggle holiday if provided
        if data.toggle_holiday is not None:
            scenario_df["IsHoliday_x"] = data.toggle_holiday

        # Scenario prediction
        scenario_pred = float(pipeline.predict(scenario_df)[0])

        # Delta & Impact
        delta = scenario_pred - base_pred
        impact = (delta / base_pred) * 100 if base_pred != 0 else 0

        return {
            "Base_Predicted_Sales": base_pred,
            "Scenario_Predicted_Sales": scenario_pred,
            "Delta": delta,
            "Impact": f"{impact:+.2f}%",
            "Fuel_Price_Adjusted": float(scenario_df["Fuel_Price"].iloc[0]),
            "Sales_MA3_Adjusted": float(scenario_df["Sales_MA3"].iloc[0]),
            "Sales_MA7_Adjusted": float(scenario_df["Sales_MA7"].iloc[0]),
            "IsHoliday_Adjusted": int(scenario_df["IsHoliday_x"].iloc[0])
        }

    except Exception as e:
        import traceback
        return {"error": str(e), "trace": traceback.format_exc()}


# --------------------------
# Start FastAPI in background
# --------------------------
def start_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=start_uvicorn, daemon=True)
thread.start()

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print("Public URL:", public_url)
print(f"Swagger UI: {public_url}/docs")


Requirement already satisfied: fastapi in /usr/local/lib/python3.12/dist-packages (0.116.1)

Requirement already satisfied: uvicorn in /usr/local/lib/python3.12/dist-packages (0.35.0)

Requirement already satisfied: nest_asyncio in /usr/local/lib/python3.12/dist-packages (1.6.0)

Requirement already satisfied: pyngrok in /usr/local/lib/python3.12/dist-packages (7.3.0)

Requirement already satisfied: joblib in /usr/local/lib/python3.12/dist-packages (1.5.2)

Requirement already satisfied: starlette<0.48.0,>=0.40.0 in /usr/local/lib/python3.12/dist-packages (from fastapi) (0.47.3)

Requirement already satisfied: pydantic!=1.8,!=1.8.1,!=2.0.0,!=2.0.1,!=2.1.0,<3.0.0,>=1.7.4 in /usr/local/lib/python3.12/dist-packages (from fastapi) (2.11.7)

Requirement already satisfied: typing-extensions>=4.8.0 in /usr/local/lib/python3.12/dist-packages (from fastapi) (4.15.0)

Requirement already satisfied: click>=7.0 in /usr/local/lib/python3.12/dist-packages (from uvicorn) (8.2.1)

Requirement already satisfied: h11>=0.8 in /usr/local/lib/python3.12/dist-packages (from uvicorn) (0.16.0)

Requirement already satisfied: PyYAML>=5.1 in /usr/local/lib/python3.12/dist-packages (from pyngrok) (6.0.2)

Requirement already satisfied: annotated-types>=0.6.0 in /usr/local/lib/python3.12/dist-packages (from pydantic!=1.8,!=1.8.1,!=2.0.0,!=2.0.1,!=2.1.0,<3.0.0,>=1.7.4->fastapi) (0.7.0)

Requirement already satisfied: pydantic-core==2.33.2 in /usr/local/lib/python3.12/dist-packages (from pydantic!=1.8,!=1.8.1,!=2.0.0,!=2.0.1,!=2.1.0,<3.0.0,>=1.7.4->fastapi) (2.33.2)

Requirement already satisfied: typing-inspection>=0.4.0 in /usr/local/lib/python3.12/dist-packages (from pydantic!=1.8,!=1.8.1,!=2.0.0,!=2.0.1,!=2.1.0,<3.0.0,>=1.7.4->fastapi) (0.4.1)

Requirement already satisfied: anyio<5,>=3.6.2 in /usr/local/lib/python3.12/dist-packages (from starlette<0.48.0,>=0.40.0->fastapi) (4.10.0)

Requirement already satisfied: idna>=2.8 in /usr/local/lib/python3.12/dist-packages (from anyio<5,>=3.6.2->starlette<0.48.0,>=0.40.0->fastapi) (3.10)

Requirement already satisfied: sniffio>=1.1 in /usr/local/lib/python3.12/dist-packages (from anyio<5,>=3.6.2->starlette<0.48.0,>=0.40.0->fastapi) (1.3.1)

INFO:     Started server process [13744]

INFO:     Waiting for application startup.

INFO:     Application startup complete.

ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use

INFO:     Waiting for application shutdown.

INFO:     Application shutdown complete.

Public URL: NgrokTunnel: "https://526bdd857337.ngrok-free.app" -> "http://localhost:8000"

Swagger UI: NgrokTunnel: "https://526bdd857337.ngrok-free.app" -> "http://localhost:8000"/docs